In [7]:
# !pip install ultralytics

In [2]:
# kütüphaneleri yüklüyoruz
from ultralytics import YOLO
import cv2
import math

In [3]:
# kameradan video alacağız
cap = cv2.VideoCapture(0)  # kamerayı açar - 1den fazla kamera varsa o yerine kamera indeksi yazılabilir.-
cap.set(3, 1280)           # genişliği 1280 piksele ayarlar
cap.set(4, 720)            # yüksekliği 720 piksele ayarlar

True

In [4]:
# YOLOv8 önceden eğitilmiş nesne tanıma modelini yüklüyoruz.
model = YOLO("yolo-Weights/yolov8n.pt")

In [5]:
# 80 nesne katogerisini temsil ediyor
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]
# pencere boyutunu ayarla

#cv2.namedWindow('Webcam', cv2.WINDOW_NORMAL)
#cv2.resizeWindow('Webcam', 1280, 720)  # Adjust the size according to your preference


In [6]:
# kameradan alınan görüntü ile nesne tespiti yapıp sonuçları görünüt üzerinden gösterir.

while True:   # sürekli tekrar eden döngü başlatılıyor. ç tuşuna basınca döngüden çıkar
    success, img = cap.read()  # kameradan bir görüntü karesi alınıyor.
    results = model(img, stream=True)   # alınan görüntü YOLO-v8 modeline gönderiliyor ve tespit yapılıyor

    
    for r in results:   # görüntü tespiti yapıldıkça ilerlemeye devam ediyor.
        boxes = r.boxes   # nesnenin etrafındaki kutunun bilgilerini alıyor.

        for box in boxes:  # birden fazla nesne kutu olamsı durumunda her kutu için işlem yapıyor.
            # kutuyu oluşturuyoruz
            x1, y1, x2, y2 = box.xyxy[0]  # kutunun sol üst ve sağ alt köşesinin kordinatları alınıyor.
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # kordinatlar int'a dönüştürülür

            # tespit edilen görüntünün etrafına RGB ayarlarında kutu çiziyor
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # doğruluk oranı nı yazdırıyoruz
            confidence = math.ceil((box.conf[0]*100))
            
            # doğruluk yüzdesine göre renk seçme
            if confidence > 80:
                color = (0, 255, 0)  # Yeşil
            elif confidence > 50:
                color = (255, 255, 0)  # Sarı
            else:
                color = (255, 0, 0)  # Kırmızı
                
            # Sınırlayıcı kutu çizme
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

            # tespit edilen nesnenin sınıfınıın indeksini alıyor.
            cls = int(box.cls[0])

            # nesne detayları
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX  # yazı fontu
            fontScale = 1   # yazı boyutu
            color = (255, 0, 0)  # yazı rengi
            thickness = 2   # iki piksel kalınlığında mwtin çizwer

            # tespit edilen nesnenin sınıf adı yazdırılıyor
            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)
            cv2.putText(img, f"Dogruluk: {confidence}%", (x1, y1-20), font, fontScale, (0, 0, 255), thickness)

    # görüntüleme 
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('ç'):
        break

cap.release()
cv2.destroyAllWindows()


Confidence ---> % 87
Class name --> person
Confidence ---> % 46
Class name --> person
Confidence ---> % 31
Class name --> bottle
0: 384x640 2 persons, 1 bottle, 85.3ms
Speed: 6.0ms preprocess, 85.3ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 90
Class name --> person
0: 384x640 1 person, 52.1ms
Speed: 2.0ms preprocess, 52.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 90
Class name --> person
0: 384x640 1 person, 51.2ms
Speed: 2.0ms preprocess, 51.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
0: 384x640 1 person, 50.0ms
Speed: 1.0ms preprocess, 50.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 85
Class name --> person
0: 384x640 1 person, 51.4ms
Speed: 3.0ms preprocess, 51.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 82
Class name --> person
0: 384x640 1 p


Confidence ---> % 85
Class name --> person
0: 384x640 1 person, 46.0ms
Speed: 2.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 85
Class name --> person
0: 384x640 1 person, 45.5ms
Speed: 2.0ms preprocess, 45.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
0: 384x640 1 person, 43.0ms
Speed: 1.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
0: 384x640 1 person, 44.1ms
Speed: 2.0ms preprocess, 44.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
0: 384x640 1 person, 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 84
Class name --> person
0: 384x640 1 person, 48.0ms
Speed: 1.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1,


Confidence ---> % 92
Class name --> person
0: 384x640 1 person, 44.3ms
Speed: 1.0ms preprocess, 44.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
0: 384x640 1 person, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 91
Class name --> person
0: 384x640 1 person, 44.0ms
Speed: 2.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
0: 384x640 1 person, 43.0ms
Speed: 1.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 91
Class name --> person
Confidence ---> % 35
Class name --> suitcase
Confidence ---> % 26
Class name --> suitcase
0: 384x640 1 person, 2 suitcases, 43.3ms
Speed: 2.0ms preprocess, 43.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
Confiden

Speed: 2.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 91
Class name --> person
Confidence ---> % 26
Class name --> bed
0: 384x640 1 person, 1 bed, 43.2ms
Speed: 1.0ms preprocess, 43.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
0: 384x640 1 person, 44.0ms
Speed: 2.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 91
Class name --> person
Confidence ---> % 34
Class name --> scissors
0: 384x640 1 person, 1 scissors, 44.2ms
Speed: 2.0ms preprocess, 44.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
0: 384x640 1 person, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
0: 384x640 1 person, 43.5ms
Speed: 2.0ms preprocess, 43.5ms inference, 0.0ms po


Confidence ---> % 91
Class name --> person
Confidence ---> % 39
Class name --> remote
0: 384x640 1 person, 1 remote, 44.1ms
Speed: 2.0ms preprocess, 44.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
Confidence ---> % 39
Class name --> remote
0: 384x640 1 person, 1 remote, 44.0ms
Speed: 1.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
0: 384x640 1 person, 44.2ms
Speed: 1.0ms preprocess, 44.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 90
Class name --> person
Confidence ---> % 28
Class name --> remote
0: 384x640 1 person, 1 remote, 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 91
Class name --> person
Confidence ---> % 32
Class name --> remote
0: 384x640 1 person, 1 remote, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inferenc

...

Speed: 1.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
Confidence ---> % 62
Class name --> person
0: 384x640 2 persons, 45.5ms
Speed: 2.0ms preprocess, 45.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
Confidence ---> % 43
Class name --> person
0: 384x640 2 persons, 46.0ms
Speed: 1.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
Confidence ---> % 54
Class name --> person
0: 384x640 2 persons, 44.0ms
Speed: 3.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 85
Class name --> person
Confidence ---> % 39
Class name --> person
Confidence ---> % 39
Class name --> person
0: 384x640 3 persons, 44.5ms
Speed: 2.0ms preprocess, 44.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confide


Confidence ---> % 88
Class name --> person
Confidence ---> % 52
Class name --> person
0: 384x640 2 persons, 52.2ms
Speed: 2.0ms preprocess, 52.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
Confidence ---> % 52
Class name --> person
0: 384x640 2 persons, 52.1ms
Speed: 2.0ms preprocess, 52.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 88
Class name --> person
Confidence ---> % 40
Class name --> person
Confidence ---> % 34
Class name --> person
0: 384x640 3 persons, 53.0ms
Speed: 1.0ms preprocess, 53.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
Confidence ---> % 37
Class name --> person
Confidence ---> % 32
Class name --> person
0: 384x640 3 persons, 56.5ms
Speed: 3.0ms preprocess, 56.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
Confidence ---> % 

Speed: 1.0ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
Confidence ---> % 52
Class name --> person
0: 384x640 2 persons, 44.1ms
Speed: 1.0ms preprocess, 44.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
Confidence ---> % 52
Class name --> person
0: 384x640 2 persons, 44.0ms
Speed: 2.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 85
Class name --> person
Confidence ---> % 51
Class name --> person
0: 384x640 2 persons, 46.1ms
Speed: 1.0ms preprocess, 46.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 85
Class name --> person
Confidence ---> % 52
Class name --> person
0: 384x640 2 persons, 45.0ms
Speed: 2.0ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 84
Class name --> person
Confide


Confidence ---> % 87
Class name --> person
Confidence ---> % 39
Class name --> person
Confidence ---> % 26
Class name --> person
0: 384x640 3 persons, 48.0ms
Speed: 1.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 86
Class name --> person
Confidence ---> % 45
Class name --> person
0: 384x640 2 persons, 46.0ms
Speed: 1.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 87
Class name --> person
Confidence ---> % 72
Class name --> person
0: 384x640 2 persons, 46.0ms
Speed: 2.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 88
Class name --> person
Confidence ---> % 64
Class name --> person
0: 384x640 2 persons, 46.0ms
Speed: 3.2ms preprocess, 46.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 88
Class name --> person
Confidence ---> % 31
Class name --> person
0: 384x640 2 perso


Confidence ---> % 90
Class name --> person
Confidence ---> % 80
Class name --> person
0: 384x640 2 persons, 45.7ms
Speed: 1.0ms preprocess, 45.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 90
Class name --> person
Confidence ---> % 85
Class name --> person
0: 384x640 2 persons, 45.6ms
Speed: 2.2ms preprocess, 45.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 91
Class name --> person
Confidence ---> % 65
Class name --> person
0: 384x640 2 persons, 45.0ms
Speed: 1.1ms preprocess, 45.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 92
Class name --> person
Confidence ---> % 92
Class name --> person
0: 384x640 2 persons, 46.7ms
Speed: 1.0ms preprocess, 46.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

Confidence ---> % 93
Class name --> person
Confidence ---> % 91
Class name --> person
0: 384x640 2 persons, 46.5ms
Speed: 1.4ms preprocess, 46.5ms 